In [2]:
import re
#import functions_is as fis

import numpy as np
import string

In [3]:
#BinToDec() == int("000110", 2) # bin in int
#’a = {0:10b}’.format(a) #bina

def listToString(list1):
    return (' '.join(map(str, list1)))

def listToString_plus(list1):
    return ('+'.join(map(str, list1)))



def Bin2Pos(x):
    '''принимаю one-hot значение(бинарный формат) в виде строки
    возвращаю позицию единицы
    "00001" == 0
    '''
    #was named DectoMyNum(x):
    x = int(x,2)
    for i in range(10):
        if x == pow(2, i): return(i)
    return(0)

def oh2i(vec): 
    '''convert one-hot (integer vector) to integer number. 
    Simply return position of 1 in vector  '''
    vec = list(map(round, vec)) # for float input convert into int
    for i in range(len(vec)): 
        if vec[i] == 1: return(i)
    return(0) 

def RowDimToRange(y,dim):
    '''get pow of 2 and dim - return range((dim*i+1),(dim*(i+1)), from 0'''
    for i in range(10):
        if y == pow(2, i): return( range((dim*i),(dim*(i+1))))
    return(0)

def RowDimToRange_2(y,dim):
    '''get integer and dim - return range((dim*(i-1)+1),(dim*i)), from 0'''
    for i in range(0, 10):
        if y == i: return( range((dim*i),(dim*(i+1))))
    return(0)

def selectFromDF(dframe, left, right, dim):
    '''get selection from np.array, according to dimension
    уномение y*dframe*x, y - м.б. 0, когда нужно умножение только с одним множетелем
    добавь вариант когда х - ноль
    '''
    if not(sum(left)): range_left = range(dframe.shape[0]) # y = zero, so get all rows
    else:
        #row <- DectoMyNum(y)
        leftd_r = oh2i(left) # analize were 1 position in left term
        range_left = RowDimToRange_2(leftd_r, dim)
  
    if not(sum(right)): range_right = range(dframe.shape[1]) # x = zero, so get all columns
    else:
        range_right = oh2i(right) # num of 1 in vec
  
    dframe = dframe[range_left,range_right]
    return(dframe.tolist())

def Fcomb(df_str, df_col, dim): 
    '''формируем столбец результата умножения строки на столбец для текстовых переменных
    считаю что размерность матрицы и ячеек матрицы - совпадают. row and col - linear
    на вход получаю 2 списка. Выдаю список'''
    Rdf = [i for i in range(dim)]
    txt = [i for i in range(dim)]
    for m in range(dim):    # in cols
        for i in range(dim):  # in rows
            range_str = RowDimToRange_2(0,dim)
            range_col = RowDimToRange_2(i,dim) 
            # column data is complex, so there is range and recounting in range
            txt[i] = str(df_str[range_str[i]]) + "*" + str(df_col[range_col[m]]) 
            # range set through i and position in range through m
   
        Rdf[m] = "(" +  listToString_plus(txt) + ")"
    return(Rdf)

def m_create(letter, dim):
    '''create array with numbered simbols, from letter'''
    m_t = np.empty((dim*dim,dim), dtype = 'U6')
    for j in range(dim):
        for i in range(dim):
            for d in range(dim):
                m_t[i*dim+d,j] = '{}{}{}_{}'.format(string.ascii_lowercase[letter], j, i, d)
    return m_t 

def l_create(dim): return [i for i in range(dim)]

def l_create_0oh(dim):
    '''return one-hot zero vector '''
    zero = [1]
    zero = zero + [0 for i in range(dim-1)]
    return zero

def ll_create_0oh(l_num, dim):
    list_l = []
    for m in range(l_num):
        list_l.append(l_create_0oh(dim))
    return list_l
    
def lm_create(m_num, dim):
    '''return list of np matrises. Contain chars in one-hot style'''
    list_m = []
    for m in range(m_num):
        list_m.append(m_create(m, dim))
    return list_m


In [121]:
def note_simplify_id(note, list_l, list_M, dim):
    '''Core function for character multiplication 
    in IRM identification'''
    i = len(note)-1 # № of M in note
    res = 0
  
    while note[i][0] != "M": i = i - 1
    numOfList_1 = int(note[i+1][1:len(note[i+1])]) - 1
    numOfList_2 = int(note[i+2][1:len(note[i+2])]) - 1
    # - 1 needs because lists in note enumeted from 1
    M = list_M[len(list_M)-1]
    list_1 = list_l[numOfList_1]
    if i + 2 <= len(note):
        # there are 2 lists or no Matr
        list_2 = list_l[numOfList_2]
        # get info about vectors in list positions
        list_1_type = isinstance(list_1[1], str)
        list_2_type = isinstance(list_2[1], str)
        # caculate result in different situations
        if not(list_1_type) and not(list_2_type): 
            # both numerical # "types: FALSE FALSE"
            res = selectFromDF(M, list_2, list_1, dim) 
    
        if (list_1_type == False) and (list_2_type == True): 
            # 1 - numerical, 2 - str
            res = selectFromDF(M, [0], list_1, dim) 
            res = Fcomb(list_2, res, dim) 
    
        if list_1_type and list_2_type: 
            # both str      
            # get M by strings(oh, wide = dim) and throgth Fcomb form one column
            res_comb_t = []
            res = []
            for k in range(dim):
                v = []
                for ii in range(dim):
                    for j in RowDimToRange_2(k,dim):
                        v = np.hstack(v, M[j,ii])
                print(v)
                res = Fcomb(list_1, v, dim)
                res_comb_t = np.hstack(res_comb_t, res)
                # w with lists so have row in output 
            res_comb_t
            res = Fcomb(list_2, res_comb_t, dim) 
            # M1*res_2 #vec(2)*vec(4) -> vec(2)

    note[i] = "l{}".format(numOfList_1+1)
    if i + 3 <= len(note): note = note[0:(i+1)]       
    note = note[0:(i+1)] # del i+1 item
    return note, res, numOfList_1, numOfList_2


In [120]:
def shell_find(f_text_v, mark=0):
    '''return coordinates of upper layer brakets in phrase'''
    Term1_r_brct_ind, Term1_l_brct_ind, Term1_r_brct_cnt, Term1_l_brct_cnt = 0, 0, 0, 0
    Term1_l_brct_found = 0
    while mark < len(f_text_v):         
        if f_text_v[mark] == '(': 
            Term1_l_brct_cnt = Term1_l_brct_cnt + 1
            if not Term1_l_brct_found:
                Term1_l_brct_ind = mark 
                Term1_l_brct_found = 1
        if f_text_v[mark] == ')':
            if Term1_r_brct_cnt >= Term1_l_brct_cnt: return -1, -1 # like ...)*()
            Term1_r_brct_cnt = Term1_r_brct_cnt + 1
            Term1_r_brct_ind = mark    
        if (Term1_l_brct_cnt != 0) and (Term1_r_brct_cnt == Term1_l_brct_cnt): break
        mark = mark + 1
    if Term1_l_brct_cnt != Term1_r_brct_cnt: return -1, -1
    else: return Term1_l_brct_ind, Term1_r_brct_ind

def rbracket_find(f_text_v, mark=0):
    '''return coordinates of upper layer right braket in phrase'''
    Term1_r_brct_ind, Term1_l_brct_ind, Term1_r_brct_cnt, Term1_l_brct_cnt = 0, 0, 0, 0
    Term1_l_brct_found = 0
    while mark < len(f_text_v):         
        if f_text_v[mark] == '(': 
            Term1_l_brct_cnt = Term1_l_brct_cnt + 1
            if not Term1_l_brct_found:
                Term1_l_brct_ind = mark 
                Term1_l_brct_found = 1
        if f_text_v[mark] == ')':
            if Term1_r_brct_cnt >= Term1_l_brct_cnt: return -1 # like ...)*()
            Term1_r_brct_cnt = Term1_r_brct_cnt + 1
            Term1_r_brct_ind = mark    
        if (Term1_l_brct_cnt != 0) and (Term1_r_brct_cnt == Term1_l_brct_cnt): break
        mark = mark + 1
    if Term1_l_brct_cnt != Term1_r_brct_cnt: return -1
    else: return Term1_r_brct_ind

    
def get_monomials(f_text):
    '''return list of monoms (was named substr_monomial)'''
    doc = re.sub("\+", ' ', f_text) 
    doc = re.sub("\)|\(", ' ', doc)
    vals = doc.split()
    vals  = list(dict.fromkeys(vals))
    vals.sort()
    return(vals)



In [118]:
def polynomial_openbrackets(f_text_v):
    '''It opens brackets in polynomial'''
    item_lgth = 5 
    # like a00_0. Better to detect item_lgth
    Term1_r_brct_ind, Term1_l_brct_ind, Term1_r_brct_cnt, Term1_l_brct_cnt = [0 for _ in range(4)]
    Term2_r_brct_ind, Term2_l_brct_ind, Term2_r_brct_cnt, Term2_l_brct_cnt = [0 for _ in range(4)]
    err_code, mark = 0, 0
    store_1, store_2, str_sum, str_new = ["" for _ in range(4)]
    if(not re.search('\(', f_text_v)): return(f_text_v)
  
    while(mark < len(f_text_v)): 
        # better stop condition?
        Term1_l_brct_ind, Term1_r_brct_ind = shell_find(f_text_v, mark)
        mark = Term1_r_brct_ind
        if(max(Term1_l_brct_ind, Term1_r_brct_ind) == 0):
            #break  # there are no brackets in phrase
            if(str_new == ""): return(f_text_v) # first time on this recursion lvl
            else: return(str_new) # there was result on this lvl

        if(max(Term1_l_brct_ind, Term1_r_brct_ind) == -1):
            err_code <- 1
            break
    
        if( (Term1_r_brct_ind < len(f_text_v)-1) and (f_text_v[mark+1] == '*') ):
            # ()*smth: ()*a or ()*() 
            if(f_text_v[mark+2] != "("): 
                # monomial next after bracket
                Term2_l_brct_ind = mark+2
                Term2_r_brct_ind = mark+2+item_lgth
                mark = Term2_r_brct_ind
            else: 
                # another () next after first bracket
                Term2_l_brct_ind = mark+3
                # +3 because I whant to get the brackets content
                Term2_r_brct_ind = rbracket_find(f_text_v, mark+2)
                mark = Term2_r_brct_ind
            # get what in brackets, without "(" ")"
            store_1 = polynome_openbrackets(f_text_v[Term1_l_brct_ind+1:Term1_r_brct_ind]) 
            store_2 = polynome_openbrackets(f_text_v[Term2_l_brct_ind:Term2_r_brct_ind])
            # get valute next after bracket # even if it is ()
            store_1 = get_monomials(store_1)
            store_2 = get_monomials(store_2)
            mult = [] 
            # monomial multiplication
            for i in range(len(store_1)):
                for j in range(len(store_2)): mult.append(store_1[i] + "*" + store_2[j])
            
            str_sum = "+".join(mult)
            # monomial summation
            if(str_new == ""): str_new = str_sum 
            else: str_new = "+".join([str_new, str_sum])  
            f_text_v = f_text_v[0:Term1_l_brct_ind] + f_text_v[Term2_r_brct_ind+1:len(f_text_v)]
            # remove used part
            mark = max(mark - Term2_r_brct_ind, 0)
    
        elif( (Term1_l_brct_ind > 0) and (f_text_v[Term1_l_brct_ind-1] == '*') ): 
            # a*() 
            # encoding: Term2*(Term1)
            Term2_l_brct_ind = Term1_l_brct_ind-1-item_lgth
            Term2_r_brct_ind = Term1_l_brct_ind-1
            # get what in brackets, without "(" ")"
            store_1 = polynome_openbrackets(f_text_v[Term1_l_brct_ind+1:Term1_r_brct_ind]) 
            store_2 = polynome_openbrackets(f_text_v[Term2_l_brct_ind:Term2_r_brct_ind])
            # get valute next after bracket # even if it ()
            store_1 = get_monomials(store_1)
            store_2 = get_monomials(store_2)
            mult = [] 
            # monomial multiplication
            for i in range(len(store_1)):
                for j in range(len(store_2)): mult.append(store_1[i] + "*" + store_2[j])
            
            str_sum = "+".join(mult) 
            # monomial summation
            if(str_new == ""): str_new = str_sum 
            else: str_new = "+".join([str_new, str_sum])
            f_text_v = f_text_v[0:Term2_l_brct_ind] + f_text_v[Term1_r_brct_ind+1:len(f_text_v)]
            # remove used part
            mark = max(mark - Term1_r_brct_ind, 0)

        else: 
            # +()+ case
            store_1 = polynome_openbrackets(f_text_v[Term1_l_brct_ind+1:Term1_r_brct_ind]) 
            if(str_new == ""): str_new = store_1 
            else: str_new = "+".join([str_new, store_1])
      
            f_text_v = f_text_v[0:Term1_l_brct_ind] + f_text_v[Term1_r_brct_ind+1:len(f_text_v)]
            mark = max(mark - Term1_r_brct_ind, 0)

    return(str_new)

